In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import plotly.express as px
import plotly.graph_objects as go
import importlib
import performance_shr
importlib.reload(performance_shr)
from performance_shr import convert_to_float

In [8]:
default_threshold = 0.10   # eg 0.1 = 10%
file_dir = 'D:/Astro/Images/M42 Orion Nebula/2024-12-11 Orion 800mm 10s/LIGHTS'
file_name = 'ImageMetaData.csv'
file_path = os.path.join(file_dir, file_name)
converters_dict = {
    'FilePath': str,   
    'Gain': convert_to_float,   
    'DetectedStars': convert_to_float,   
    'HFR': convert_to_float,  
    'HFRStDev': convert_to_float,  
    'FWHM': convert_to_float,  
    'Eccentricity': convert_to_float,  
    'FocuserPosition': convert_to_float,  
    'Airmass': convert_to_float,  
    'MountRA': convert_to_float,  
    'MountDec': convert_to_float,  
    }

In [10]:
data = pd.read_csv(file_path, converters=converters_dict)
data['FileName'] = data['FilePath'].apply(lambda x: os.path.basename(x))
data = data[data['FilePath'].str.strip() != '' ]
data

,ExposureNumber,FilePath,FilterName,ExposureStart,Duration,Binning,CameraTemp,CameraTargetTemp,Gain,Offset,...,GuidingRMSDECArcSec,FocuserPosition,FocuserTemp,RotatorPosition,PierSide,Airmass,ExposureStartUTC,MountRA,MountDec,FileName
0,0,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 20:52,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,2.4075,2024-12-11T09:52:51.6818609Z,83.863025,-5.397036,2024-12-11_20-52-51_NoFilter_400_10.00s_0000.cr3
1,1,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 20:53,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,2.4038,2024-12-11T09:53:03.3758685Z,83.863098,-5.396948,2024-12-11_20-53-03_NoFilter_400_10.00s_0001.cr3
2,2,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 20:53,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,2.4000,2024-12-11T09:53:15.5393643Z,83.863264,-5.396956,2024-12-11_20-53-15_NoFilter_400_10.00s_0002.cr3
3,3,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 20:53,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,2.3963,2024-12-11T09:53:27.7197663Z,83.863451,-5.397181,2024-12-11_20-53-27_NoFilter_400_10.00s_0003.cr3
4,4,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 20:53,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,2.3926,2024-12-11T09:53:40.1482950Z,83.863671,-5.397038,2024-12-11_20-53-40_NoFilter_400_10.00s_0004.cr3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,845,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 23:42,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,1.2150,2024-12-11T12:42:46.5986453Z,83.856833,-5.386560,2024-12-11_23-42-46_NoFilter_400_10.00s_0845.cr3
846,846,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 23:42,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,1.2147,2024-12-11T12:42:58.3634758Z,83.857152,-5.386603,2024-12-11_23-42-58_NoFilter_400_10.00s_0846.cr3
847,847,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 23:43,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,1.2143,2024-12-11T12:43:10.2798048Z,83.857297,-5.386431,2024-12-11_23-43-10_NoFilter_400_10.00s_0847.cr3
848,848,D:/Astro/Images/2024-12-11/M 42/LIGHT/2024-12-...,NoFilter,2024-12-11 23:43,10,1x1,NaN,NaN,400.0,-1,...,0,370.0,NaN,NaN,NaN,1.2139,2024-12-11T12:43:22.2554662Z,83.857633,-5.386537,2024-12-11_23-43-22_NoFilter_400_10.00s_0848.cr3


In [12]:
def filter_percentiles(df, col='HFR', top=default_threshold, bottom=default_threshold, zero=True):
    top_threshold = df[col].quantile(1 - top) if top else False
    bottom_threshold = df[col].quantile(bottom) if bottom else False
    is_na = df[col].isna()
    is_false = is_na != is_na
    is_zero = df[col] == 0 if zero else is_false
    is_top = df[col] >= top_threshold if top_threshold else is_false
    is_bottom = df[col] <= bottom_threshold if bottom_threshold else is_false
    is_filtered = is_zero | is_na | is_top | is_bottom

    fig=go.Figure()
    color_map = {True: 'red', False: 'blue'}
    fig.add_trace(go.Scatter(x=df.ExposureNumber, y=df[col], name=col, mode='markers', marker=dict(color=np.vectorize(color_map.get)(is_filtered))))
    fig.update_layout(title=f'{col} vs Exposure No.', xaxis_title='Exposure No.', yaxis_title=col)
    fig.show()    

    print(f'{col}: TopThreshold={top_threshold:.3f} n({is_top.sum()}), BottomThreshold={bottom_threshold:.3f} n({is_bottom.sum()}), na=n({is_na.sum()}), zero=n({is_zero.sum()}), filtered=n({is_filtered.sum()})\n\n')
    return is_filtered

In [16]:
print(f'**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****')
is_badHFR = filter_percentiles(data, col='HFR', top=0.10, bottom=False)
is_badHFRStDev = filter_percentiles(data, col='HFRStDev', top=0.05, bottom=0.01)
is_badFWHM = filter_percentiles(data, col='FWHM', top=0.08, bottom=0.01)
is_badEccentricity = filter_percentiles(data, col='Eccentricity', top=0.10, bottom=0.10)
is_badDetectedStars = filter_percentiles(data, col='DetectedStars', top=False, bottom=0.10)
is_rejected = is_badFWHM | is_badHFR | is_badFWHM | is_badHFRStDev | is_badEccentricity | is_badDetectedStars
print(f'\nTotal Rejected: n({is_rejected.sum()}/{len(data)}, Total Images: n({len(data) - is_rejected.sum()})')


**** Rejecting Images based on top and bottom percentiles for HFR, FWHM, Eccentricity and #Stars Dectected ****


HFR: TopThreshold=3.053 n(85), BottomThreshold=0.000 n(0), na=n(0), zero=n(0), filtered=n(85)




HFRStDev: TopThreshold=0.423 n(43), BottomThreshold=0.255 n(9), na=n(0), zero=n(0), filtered=n(52)




FWHM: TopThreshold=7.781 n(68), BottomThreshold=6.522 n(9), na=n(10), zero=n(0), filtered=n(87)




Eccentricity: TopThreshold=0.544 n(84), BottomThreshold=0.267 n(84), na=n(10), zero=n(0), filtered=n(178)




DetectedStars: TopThreshold=0.000 n(0), BottomThreshold=170.000 n(86), na=n(0), zero=n(0), filtered=n(86)



Total Rejected: n(238/850, Total Images: n(612)


In [14]:
# Create the 'rejected' directory at the same level as file_dir if it doesn't exist 
parent_dir = os.path.dirname(file_dir) 
rejected_dir = os.path.join(parent_dir, 'Rejected') 
os.makedirs(rejected_dir, exist_ok=True)

In [15]:
rejects = data[is_rejected].FileName
for filename in rejects:
    src_path = os.path.join(file_dir, filename)
    dest_path = os.path.join(rejected_dir, filename)
    if os.path.exists(src_path): 
        shutil.move(src_path, dest_path) 
        print(f'Moved: {filename}')

print('File moving complete.')

Moved: 2024-12-11_20-52-51_NoFilter_400_10.00s_0000.cr3
Moved: 2024-12-11_20-53-03_NoFilter_400_10.00s_0001.cr3
Moved: 2024-12-11_20-53-15_NoFilter_400_10.00s_0002.cr3
Moved: 2024-12-11_20-53-27_NoFilter_400_10.00s_0003.cr3
Moved: 2024-12-11_20-53-40_NoFilter_400_10.00s_0004.cr3
Moved: 2024-12-11_20-53-52_NoFilter_400_10.00s_0005.cr3
Moved: 2024-12-11_20-54-04_NoFilter_400_10.00s_0006.cr3
Moved: 2024-12-11_20-54-16_NoFilter_400_10.00s_0007.cr3
Moved: 2024-12-11_20-54-28_NoFilter_400_10.00s_0008.cr3
Moved: 2024-12-11_20-54-41_NoFilter_400_10.00s_0009.cr3
Moved: 2024-12-11_20-54-53_NoFilter_400_10.00s_0010.cr3
Moved: 2024-12-11_20-55-05_NoFilter_400_10.00s_0011.cr3
Moved: 2024-12-11_20-55-17_NoFilter_400_10.00s_0012.cr3
Moved: 2024-12-11_20-55-29_NoFilter_400_10.00s_0013.cr3
Moved: 2024-12-11_20-55-41_NoFilter_400_10.00s_0014.cr3
Moved: 2024-12-11_20-55-54_NoFilter_400_10.00s_0015.cr3
Moved: 2024-12-11_20-56-06_NoFilter_400_10.00s_0016.cr3
Moved: 2024-12-11_20-56-18_NoFilter_400_10.00s_0